In [1]:
# baseline cnn model
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from os import listdir
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# load train and test dataset
def load_dataset(pathImages, pathLabeled):
    onlyfiles = [f for f in os.listdir(pathImages) if os.path.isfile(os.path.join(pathImages, f))]
    
    train_files = []
    y_train = []
    i=0
    for _file in onlyfiles:
        train_files.append(_file)

    image_width = 1000
    image_height = 800
    ratio = 2

    image_width = int(image_width / ratio)
    image_height = int(image_height / ratio)

    channels = 3
    nb_classes = 1

    dataset = np.ndarray(shape=(len(train_files), channels, image_height, image_width),
                         dtype=np.float32)
    i = 0
    for _file in train_files:
        img = load_img(pathImages + "/" + _file)  # this is a PIL image
        img.thumbnail((image_width, image_height))
        # Convert to Numpy Array
        x = img_to_array(img)  
        x = x.reshape(-1, 3,400,500)
        # Normalize
#         x = (x - 128.0) / 128.0
        dataset[i] = x
        i += 1
        if i % 100 == 0:
            print("%d images to array" % i)
    print("All images to array!")

#    for i in dataset:
#        print(i.size)
    
    z_train, y_train = np.loadtxt(pathLabeled, delimiter=' | ', unpack = True)
    
    from sklearn.model_selection import train_test_split

    #Splitting 
    X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.4, random_state=0)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
    
#     from keras.utils import to_categorical
#     y_binary = to_categorical(y_train)
    
    print(y_train)
    print(y_test)
    print(y_val)
    print(z_train)
#     print(y_binary)
    
    return X_train, y_train, X_test, y_test

In [3]:
# test_array = np.random.rand(3,2)
# test_vector = np.random.rand(4)

# np.savez_compressed('C:/Users/Dani/data/images', a = test_array, b = test_vector)


In [15]:
# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (0), activation='relu', kernel_initializer='he_uniform', input_shape=(3, 400, 500)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None))
    model.add(Flatten())
    model.add(Dense(100, input_shape=(200000,), activation='relu', kernel_initializer='he_uniform'))
    model.summary()
    model.add(Dense(2, activation='softmax'))
    model.summary()
#   compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [5]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

In [6]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(2, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(2, 1, 2)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

In [7]:
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

In [8]:
trainX, trainY, testX, testY = load_dataset('C:\images', 'C:\labeled.txt')

100 images to array
200 images to array
300 images to array
400 images to array
500 images to array
600 images to array
700 images to array
800 images to array
900 images to array
1000 images to array
1100 images to array
1200 images to array
1300 images to array
1400 images to array
1500 images to array
1600 images to array
1700 images to array
1800 images to array
1900 images to array
2000 images to array
2100 images to array
2200 images to array
2300 images to array
2400 images to array
2500 images to array
2600 images to array
2700 images to array
2800 images to array
2900 images to array
3000 images to array
3100 images to array
3200 images to array
3300 images to array
3400 images to array
3500 images to array
3600 images to array
3700 images to array
3800 images to array
3900 images to array
4000 images to array
4100 images to array
4200 images to array
4300 images to array
4400 images to array
4500 images to array
4600 images to array
4700 images to array
4800 images to array
4

In [9]:
trainX2, trainY2, testX2, testY2 = load_dataset('C:\images\carpetaInutil', 'C:\carpetaInutil\labeled.txt')

All images to array!
Train set size: 6, Val set size: 2, Test set size: 2
[0. 0. 0. 0. 0. 0.]
[0. 0.]
[0. 0.]
[9200. 9201. 9202. 9203. 9204. 9205. 9206. 9207. 9208. 9209.]


In [10]:
print(len(trainY2))

6


In [11]:
trainX2 = np.append(trainX2, trainX, axis = 0)
trainY2 =np.append(trainY2, trainY)
testX2 =np.append(testX2, testX, axis = 0)
testY2 =np.append(testY2, testY, axis = 0)

In [12]:
print(len(trainX2))

5526


In [13]:
# run the test harness for evaluating a model
def run_test_harness(trainX, trainY, testX, testY):
    print('iniciating the load of the dataset')
    # load dataset
#     trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    print('iniciating the  preparation of pixel data')
#     trainX, testX = prep_pixels(trainX, testX)
    # evaluate model
    print('iniciating theevaluation of the model')
    scores, histories = evaluate_model(trainX, trainY)
    # learning curves
    print('iniciating the diagnostics')
    summarize_diagnostics(histories)
    # summarize estimated performance
    print('iniciating the diagnostic of performance')
    summarize_performance(scores)
    # define model
    print('iniciating definition of the model')
    model = define_model()
    # fit model
    print('iniciating the fit of the model')
    model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    # save model
    print('saving the model')
    model.save('final_model.h5')

In [16]:
# entry point, run the test harness
run_test_harness(trainX2, trainY2, testX2, testY2)

iniciating the load of the dataset
iniciating the  preparation of pixel data
iniciating theevaluation of the model
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 4, 401, 32)        32        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 201, 32)        0         
_________________________________________________________________
dense_3 (Dense)              (None, 2, 201, 100)       3300      
Total params: 3,332
Trainable params: 3,332
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 4, 401, 32)        32        
_______________________________________________

ValueError: Error when checking target: expected dense_4 to have 4 dimensions, but got array with shape (4420, 1)

In [ ]:
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=False, target_size=(800, 1000))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(-1, 3,400,500)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [ ]:
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('sample6.jpeg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	digit = model.predict_classes(img)
	print(digit[0])

In [ ]:
# entry point, run the example
run_example()